In [5]:
# conda create -n aoai python=3.10
# conda activate aoai
# pip install -r requirements.txt

In [6]:
import os
from openai import AzureOpenAI
from IPython.display import Markdown
from dotenv import load_dotenv

load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.environ["AZURE_OPENAI_ENDPOINT"],
    api_key = os.environ["AZURE_OPENAI_API_KEY"],
    api_version = os.environ["OPENAI_API_VERSION"],
  )

deployment = os.environ["AZURE_OPENAI_GPT_MODEL"]

# add your completion code
prompt = "完成下面的内容： 从前有一个"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages)

# Markdown response
Markdown(completion.choices[0].message.content)

勇敢的小男孩，他住在一个小村庄里。每天他都会去森林探险，发现各种奇奇怪怪的事物。有一天，他遇到了一只神秘的精灵，精灵告诉他一个秘密。他依靠这个秘密帮助了村庄，并成为了村庄的英雄。 从此以后，他的名字被人们传颂，成为村庄的英雄。

In [7]:

prompt = "向我展示 5 份菜谱及其做法，其中包含以下配料：鸡肉、土豆和胡萝卜。每个食谱列出所用的所有配料"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# Markdown response
Markdown(completion.choices[0].message.content)

1. 土豆鸡肉煲
配料：鸡肉块、土豆、胡萝卜、洋葱、蘑菇、鸡高汤、盐、胡椒粉、食用油
做法：将鸡肉块煎至金黄色，加入洋葱和蘑菇炒香，倒入鸡高汤，加入土豆和胡萝卜，小火炖煮至熟透，最后加入盐和胡椒粉调味即可。

2. 烤鸡胸肉配土豆胡萝卜泥
配料：鸡胸肉、土豆、胡萝卜、牛奶、黄油、盐、黑胡椒、橄榄油
做法：将鸡胸肉用盐和黑胡椒腌制，烤至金黄色；土豆和胡萝卜切块煮熟，加入牛奶和黄油，捣成泥状，用橄榄油拌匀，作为鸡肉的搭配。

3. 蜜汁鸡肉土豆胡萝卜炒
配料：鸡肉丁、土豆丁、胡萝卜丁、蜂蜜、生抽、料酒、盐、食用油
做法：鸡肉丁用蜂蜜、生抽、料酒腌制，土豆和胡萝卜丁焯水备用，锅中加油热后倒入鸡肉炒至变色，加入土豆和胡萝卜丁炒熟，最后加入盐调味即可。

4. 土豆胡萝卜烧鸡肉
配料：鸡肉、土豆、胡萝卜、葱姜蒜、料酒、生抽、盐、白糖

In [8]:

no_recipes = input("食谱数量（例如，5： ")

ingredients = input("配料清单（例如，鸡肉、土豆和胡萝卜： ")

# interpolate the number of recipes into the prompt an ingredients
prompt = f"向我展示包含以下配料的菜肴的{no_recipes}食谱： {ingredients}。每个食谱列出所用的所有配料"
messages = [{"role": "user", "content": prompt}]  

# make completion
completion = client.chat.completions.create(model=deployment, messages=messages, max_tokens=600)

# Markdown response
Markdown(completion.choices[0].message.content)

食谱1：草莓牛肉沙拉
配料：
- 牛肉
- 草莓
- 生菜
- 红洋葱
- 橄榄油
- 醋
- 盐
- 黑胡椒

食谱2：草莓牛肉卷
配料：
- 牛肉
- 草莓
- 面粉卷
- 奶油芝士
- 生菜叶
- 红洋葱
- 盐
- 黑胡椒

对于测试运行代码，可能看起来像这样：

```output
食谱数量（例如，5）：3
配料列表（例如，鸡肉、土豆和胡萝卜）：牛奶，草莓

- 草莓奶昔：牛奶、草莓、糖、香草精、冰块
- 草莓蛋糕：牛奶、面粉、发酵粉、糖、盐、无盐黄油、草莓、打发奶油       
- 草莓牛奶：牛奶、草莓、糖、香草精
```

### 通过添加过滤器和购物清单进行改进

我们现在有一个能够生成食谱的工作程序，并且它非常灵活，因为它依赖于用户的输入，不仅限于食谱数量，还有使用的配料。

为了进一步改进它，我们希望添加以下功能：

- **过滤配料**。我们希望能够过滤掉自己不喜欢或对自己过敏的配料。为了实现这一改变，我们可以编辑现有的提示，并在末尾添加一个过滤条件，如下所示：

    ```python
    filter = input("过滤（例如，素食、纯素或无麸质）：")

    prompt = f"为我展示使用以下配料制作 {no_recipes} 道菜的食谱：{ingredients}。对于每个食谱，列出使用的所有配料，不含{filter}"
    ```

    在上面的代码中，我们在提示的末尾添加了 `{filter}`，并且我们还从用户那里获取了过滤值。

    现在，运行程序的示例输入可能如下所示：

    ```output    
    食谱数量（例如，5）：3
    配料列表（例如，鸡肉、土豆和胡萝卜）：洋葱，牛奶
    过滤（例如，素食、纯素或无麸质）：不含牛奶

    1. 法式洋葱汤

    配料：
    
    - 1个大洋葱，切片
    - 3杯牛肉高汤
    - 1杯牛奶
    - 6片法式面包
    - 1/4杯磨碎的帕玛森奶酪
    - 1汤匙黄油
    - 1茶匙干百里香
    - 1/4茶匙盐
    - 1/4茶匙黑胡椒
    
    烹饪步骤：
    
    1. 在大锅中，用黄油炒洋葱，直到金黄色。
    2. 加入牛肉高汤、牛奶、百里香、盐和胡椒。煮沸。
    3. 火候调至小，煨煮10分钟。
    4. 将法式面包片放在汤碗上。
    5. 舀入汤。
    6. 撒上帕玛森奶酪。

    2. 洋葱土豆汤

    配料：
    
    - 1个大洋葱，切碎
    - 2杯土豆，切丁
    - 3杯蔬菜高汤
    - 1杯牛奶
    - 1/4茶匙黑胡椒
    
    烹饪步骤：
    
    1. 在大锅中，用黄油炒洋葱，直到金黄色。
    2. 加入土豆、蔬菜高汤、牛奶和胡椒。煮沸。
    3. 火候调至小，煨煮10分钟。
    4. 热腾腾地上菜。
    
    3. 奶油洋葱汤

    配料：
    
    - 1个大洋葱，切碎
    - 3杯蔬菜高汤
    - 1杯牛奶
    - 1/4茶匙黑胡椒
    - 1/4杯多用途面粉
    - 1/2杯磨碎的帕玛森奶酪
    
    烹饪步骤：
    
    1. 在大锅中，用黄油炒洋葱，直到金黄色。
    2. 加入蔬菜高汤、牛奶和胡椒。煮沸。
    3. 火候调至小，煨煮10分钟。
    4. 在一个小碗中，搅拌面粉和帕玛森奶酪，直到平滑。
    5. 加入汤中，煨煮额外的5分钟，或直到汤变浓稠。
    ```

    如您所见，任何含牛奶的食谱都已被过滤掉。但是，如果你对乳糖不耐受，你也可能想要过滤掉含有奶酪的食谱，所以有必要明确说明。

    ```python
    
- **生成购物清单**。我们希望生成一份购物清单，考虑到我们家里已经有的东西。

    对于这个功能，我们可以尝试在一个提示中解决所有问题，或者我们可以将其分成两个提示。让我们尝试后一种方法。在这里，我们建议添加一个额外的提示，但为了使其工作，我们需要将前一个提示的结果作为后一个提示的上下文添加到其中。 

    定位代码中打印出第一个提示结果的部分，并在其下方添加以下代码：
    
    ```python
    old_prompt_result = completion.choices[0].text
    prompt = "为生成的食谱准备一份购物清单，并且请不要包括我已经有的配料。"
    
    new_prompt = f"{old_prompt_result} {prompt}"
    messages = [{"role": "user", "content": new_prompt}]
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
    
    # 打印响应
    Markdown("购物清单：")
    Markdown(completion.choices[0].message.content)
    ```

    请注意以下内容：

    - 我们通过将第一个提示的结果添加到新提示中构建了一个新提示： 
    
        ```python
        new_prompt = f"{old_prompt_result} {prompt}"
        messages = [{"role": "user", "content": new_prompt}]
        ```

    - 我们进行了一个新的请求，但也考虑了我们在第一个提示中要求的标记数量，所以这次我们说 `max_tokens` 是 1200。 

        ```python
        completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=1200)
        ```  

        运行这段代码后，我们现在得到以下输出：

        ```output
        食谱数量（例如，5）：2
        配料列表（例如，鸡肉、土豆和胡萝卜）：苹果，面粉
        过滤（例如，素食、纯素或无麸质）：糖
        食谱：
         or milk.
        
        -苹果薄饼：1杯面粉，1/2茶匙发酵粉，1/2茶匙小苏打，1/4茶匙盐，1汤匙糖，1个鸡蛋，1杯淡奶或酸奶，1/4杯融化黄油，1个青苹果，去皮和切碎
        -苹果油炸圈：1-1/2杯面粉，1茶匙发酵粉，1/4茶匙盐，1/4茶匙小苏打，1/4茶匙肉豆蔻，1/4茶匙肉桂，1/4茶匙多香料，1/4杯糖，1/4杯植物固体短ening，1/4杯牛奶，1个鸡蛋，2杯去皮苹果丝
        购物清单：
         -面粉，发酵粉，小苏打，盐，糖，鸡蛋，淡奶，黄油，苹果，肉豆蔻，肉桂，多香料 
        ```

- **关于标记长度的说明**。我们应该考虑生成所需文本所需的标记数。标记会花费资金，因此在可能的情况下，我们应该尽量节约标记的使用。例如，我们是否可以构造提示，以便我们可以使用更少的标记数？

   要更改使用的标记数，您可以使用 `max_tokens` 参数。例如，如果您想使用 100 个标记，您会这样做：

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, max_tokens=100)
    ```

- **尝试使用温度进行实验**。到目前为止，温度是我们还没有提及但对我们程序性能很重要的上下文。温度值越高，输出就越随机。相反，温度值越低，输出就越可预测。请考虑您是否希望输出中有变化。

   要更改温度，您可以使用 `temperature` 参数。例如，如果您想使用温度为 0.5，您会这样做：

    ```python
    completion = client.chat.completion.create(model=deployment, messages=messages, temperature=0.5)
    ```

   > 请注意，温度越接近 1.0，输出就会越多样化。



## 任务

对于此任务，您可以选择构建什么。

以下是一些建议：

- 调整食谱生成器应用以进一步改进它。尝试不同的温度值和提示以查看您能够得到什么。
- 开发一个“学习伙伴”。此应用程序应能够回答关于某一主题的问题，例如 Python，您可以使用像“Python 的某个特定主题是什么？”这样的提示，或者您可以使用一个提示，根据代码显示某个特定主题等。
- 历史机器人，使历史活起来，指示机器人扮演某个历史人物，并问它有关其生活和时代的问题。

## 解决方案

### 学习伙伴

- “您是 Python 语言的专家

    以以下格式建议 Python 的入门课程：
    
    格式:
    - 概念：
    - 课程的简要解释：
    - 包含解决方案的代码练习”

以上是一个起始提示，看看你如何使用它并对其进行调整。 

### 历史机器人

以下是您可以使用的一些提示：

- “您是阿伯·林肯，请用林肯可能使用的语法和词汇向我介绍您自己，用 3 句话。”
- “您是阿伯·林肯，请用林肯可能使用的语法和词汇回答：

   用 300 个单词告诉我您的最伟大成就。”

## 知识检查

概念温度的作用是什么？

1. 它控制输出的随机程度。
1. 它控制响应的大小。
1. 它控制使用的标记数量。

答：1

存储诸如 API 密钥之类的机密信息的一个好方法是什么？

1. 在代码中。
1. 在一个文件中。
1. 在环境变量中。

答：3，因为环境变量不存储在代码中，可以从代码中加载。